### Imports

In [ ]:
import json
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import codecs

### Functions

In [41]:
PATTERN = r"{}\s+=\s+JSON.parse\(\'(.*?)\'\)"

def find_match(scripts, pattern):
    for script in scripts:
        match = re.search(pattern, script.string)
        if match:
            break

    return match

def get_data(url, data_type):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    result = requests.get(url, headers=headers)
    soup = BeautifulSoup(result.content, 'html.parser')
    scripts = soup.find_all("script")

    pattern = re.compile(PATTERN.format(data_type))
    match = find_match(scripts, pattern)
    data = decode_data(match)

    return data

def decode_data(match):
    byte_data = codecs.escape_decode(match.group(1))
    json_data = json.loads(byte_data[0].decode("utf-8"))

    return json_data

## Fetch Players
* Gets players from the "top" 5 leagues in a given year

In [47]:
leagues = ['EPL', 'La_liga', 'Bundesliga', 'Serie_A', 'Ligue_1']
year = 2014
dfs = []

for i in leagues:
    dfs.append(pd.DataFrame(get_data(f'https://understat.com/league/{i}/{year}', 'playersData')))

In [1]:
# Combine the 5 leagues into 1 dataframe
total = pd.concat(dfs)

NameError: name 'pd' is not defined

In [46]:
# Output to csv file
total.to_csv("csv files/ustat_2014.csv", index=False)